### Advanced in silico drug design workshop. Olomouc, 30 January - 1 February, 2017.
### QSAR tutorial.
Dr. Pavel Polishchuk  
   
Institute of Molecular and Translational Medicine, Faculty of Medicine and Dentistry, Palacký University and University Hospital in Olomouc, Hněvotínská 1333/5, 779 00 Olomouc, Czech Republic  
http://imtm.cz  
http://qsar4u.com  
pavel_polishchuk@ukr.net  
pavlo.polishchuk@upol.cz

In [1]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors

In [2]:
# from rdkit.Chem.Draw import IPythonConsole
# from rdkit.Chem import Draw
# IPythonConsole.ipython_useSVG=True

In [3]:
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score, matthews_corrcoef
#from sklearn.externals import joblib
import joblib

#### Reading molecules and activity from SDF

In [4]:
fname = "data/logBB.sdf"

mols = []
y = []
for mol in Chem.SDMolSupplier(fname):
    if mol is not None:
        mols.append(mol)
        y.append(mol.GetIntProp("logBB_class"))

nmol = len(mols)
print ("The number of molecule is ", nmol)

The number of molecule is  321


#### Calculate descriptors (fingerprints) and convert them into numpy array

In [5]:
# generate binary Morgan fingerprint with radius 2
fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols]

In [6]:
sim_dist = np.zeros((nmol,nmol))
for imol1 in range(nmol):
    for imol2 in range(imol1+1,nmol):
        sim_dist[imol1,imol2] = DataStructs.DiceSimilarity(fp[imol1],fp[imol2])
        sim_dist[imol2,imol1] = sim_dist[imol1,imol2]

In [7]:
#import seaborn as sns
#import matplotlib.pyplot as plt
#plt.figure(figsize = (20,16))
#sns.heatmap(sim_dist, cmap="YlGnBu", annot=True, fmt=".1f")
#plt.show()

In [8]:
def rdkit_numpy_convert(fp):
    output = []
    for f in fp:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [9]:
x = rdkit_numpy_convert(fp)

In [10]:
x.shape

(321, 2048)

In [11]:
# check wether the data set is balanced
sum(y) / len(y)

0.5545171339563862

#### Split the whole set on training and test sets

In [12]:
# Set random seed to make all further calculations reproducible
seed = 42
# randomly select 20% of compounds as test set
indices = np.arange(len(y))

x_tr, x_ts, y_tr, y_ts, idx1, idx2 = train_test_split(x, y, indices, test_size=0.20, random_state=seed)

In [13]:
y_pred_ts = []
for idtest in idx2:
    numerator_   = y_tr[:] * sim_dist[idtest,idx1]
    denominator_ = sim_dist[idtest,idx1]
    y_pred_ts.append( int(round(sum(numerator_)/sum(denominator_))) )

In [14]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, y_pred_ts))
print("MCC = ", matthews_corrcoef(y_ts, y_pred_ts))
print("Kappa = ", cohen_kappa_score(y_ts, y_pred_ts))

Accuracy =  0.7230769230769231
MCC =  0.47559486560567094
Kappa =  0.3689320388349514


#### Create folds for cross-validation

In [15]:
cv = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
#cv = StratifiedKFold(n_splits=5, random_state=None)

In [16]:
# print out ids of folds
for i, (train_index, test_index) in enumerate(cv.split(x_tr, y_tr)):
    print("\nFold_" + str(i+1))
    print("TRAIN:", train_index)
    print("TEST:", test_index)


Fold_1
TRAIN: [  0   1   2   3   4   5   7   8  10  13  14  15  16  18  19  21  22  23
  24  25  26  27  28  30  32  36  37  38  39  40  42  43  44  45  46  47
  48  49  52  53  55  56  58  59  61  63  64  65  66  67  68  69  70  71
  72  73  75  76  78  79  80  82  83  84  85  86  87  88  89  90  92  93
  95  96  97  98  99 100 101 103 104 105 106 108 109 112 114 115 116 117
 118 119 120 122 123 124 125 126 127 128 129 130 131 132 134 135 136 137
 138 139 140 142 143 144 145 146 147 148 149 150 153 154 157 158 159 160
 161 162 163 165 166 167 168 169 170 171 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 193 194 195 196 197 198 199
 200 201 203 204 205 206 207 208 209 211 212 213 214 215 216 217 218 219
 220 221 222 223 224 226 229 231 232 233 234 235 236 237 238 241 242 244
 246 247 248 250 251 255]
TEST: [  6   9  11  12  17  20  29  31  33  34  35  41  50  51  54  57  60  62
  74  77  81  91  94 102 107 110 111 113 121 133 141 151 152 155 156 164
 172

#### Scale X

This step may be crucial for certain modeling approaches lke SVM.
In the case of binary fingerprints it may be less useful.

In [17]:
# obtain scale object which can be further applied to scale any data to fit the training set
scale = StandardScaler().fit(x_tr)
x_tr = scale.transform(x_tr)

In [18]:
# it is a good idea to save it for future use
joblib.dump(scale, "data/logBB_scale.pkl", compress=3)

['data/logBB_scale.pkl']

#### Search for optimal tuning parameters and build the model

In [19]:
# create grid search dictionary
param_grid = {"max_features": [x_tr.shape[1] // 10, x_tr.shape[1] // 7, x_tr.shape[1] // 5, x_tr.shape[1] // 3], 
              "n_estimators": [100, 250, 500]}

In [20]:
# setup model building
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [21]:
# run model building
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=2,
             param_grid={'max_features': [204, 292, 409, 682],
                         'n_estimators': [100, 250, 500]},
             verbose=1)

In [22]:
m.best_params_

{'max_features': 409, 'n_estimators': 250}

In [23]:
m.best_score_

0.7930618401206637

In [24]:
m.cv_results_

{'mean_fit_time': array([0.3726531 , 0.94125042, 1.86414795, 0.42256985, 1.07519417,
        2.06206431, 0.45674996, 1.12906709, 2.36443071, 0.54995298,
        1.36331973, 2.6553659 ]),
 'std_fit_time': array([0.00682861, 0.01659201, 0.00891853, 0.00425554, 0.04639724,
        0.03559016, 0.00879766, 0.01075677, 0.12509904, 0.01731615,
        0.02930505, 0.10850422]),
 'mean_score_time': array([0.01633573, 0.03574929, 0.06963158, 0.01531549, 0.03612938,
        0.0700232 , 0.01908917, 0.03630486, 0.06861701, 0.01784372,
        0.03502479, 0.06704721]),
 'std_score_time': array([0.00174539, 0.00032462, 0.00111462, 0.00031887, 0.00177013,
        0.00210879, 0.00757407, 0.00140344, 0.00053619, 0.00530548,
        0.00021557, 0.00282347]),
 'param_max_features': masked_array(data=[204, 204, 204, 292, 292, 292, 409, 409, 409, 682, 682,
                    682],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
  

In [25]:
m.cv_results_['mean_test_score']

array([0.773454  , 0.76953243, 0.78129713, 0.75392157, 0.77730015,
       0.77737557, 0.78921569, 0.79306184, 0.78514329, 0.77352941,
       0.7852187 , 0.78914027])

In [26]:
m.cv_results_['params']

[{'max_features': 204, 'n_estimators': 100},
 {'max_features': 204, 'n_estimators': 250},
 {'max_features': 204, 'n_estimators': 500},
 {'max_features': 292, 'n_estimators': 100},
 {'max_features': 292, 'n_estimators': 250},
 {'max_features': 292, 'n_estimators': 500},
 {'max_features': 409, 'n_estimators': 100},
 {'max_features': 409, 'n_estimators': 250},
 {'max_features': 409, 'n_estimators': 500},
 {'max_features': 682, 'n_estimators': 100},
 {'max_features': 682, 'n_estimators': 250},
 {'max_features': 682, 'n_estimators': 500}]

#### Save model

In [27]:
joblib.dump(m, "data/logBB_rf_morgan.pkl", compress=3)

['data/logBB_rf_morgan.pkl']

#### Predict test set compounds

In [28]:
# load scale if necessary
scale = joblib.load("data/logBB_scale.pkl")

# scale descriptors of the test set compounds
x_ts = scale.transform(x_ts)

# predict logBB class
pred_rf = m.predict(x_ts)

In [29]:
pred_rf

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1])

In [30]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred_rf))
print("MCC = ", matthews_corrcoef(y_ts, pred_rf))
print("Kappa = ", cohen_kappa_score(y_ts, pred_rf))

Accuracy =  0.7538461538461538
MCC =  0.4995357946109721
Kappa =  0.4985535197685632


#### applicability domain estimates

In [31]:
# if the model includes several ones like RF models or consensus models (or for probabilistic models)
# we can calculate consistency of predictions amongs those models and use it for estimation of applicability domain
pred_prob = m.predict_proba(x_ts)

In [32]:
# probablity
pred_prob

array([[0.892, 0.108],
       [0.208, 0.792],
       [0.3  , 0.7  ],
       [0.196, 0.804],
       [0.396, 0.604],
       [0.224, 0.776],
       [0.004, 0.996],
       [0.016, 0.984],
       [0.872, 0.128],
       [0.928, 0.072],
       [0.244, 0.756],
       [0.812, 0.188],
       [0.584, 0.416],
       [0.908, 0.092],
       [0.384, 0.616],
       [0.568, 0.432],
       [0.14 , 0.86 ],
       [0.012, 0.988],
       [0.008, 0.992],
       [0.152, 0.848],
       [0.556, 0.444],
       [0.   , 1.   ],
       [0.196, 0.804],
       [0.556, 0.444],
       [0.256, 0.744],
       [0.38 , 0.62 ],
       [0.636, 0.364],
       [0.308, 0.692],
       [0.768, 0.232],
       [0.656, 0.344],
       [0.012, 0.988],
       [0.948, 0.052],
       [0.112, 0.888],
       [0.692, 0.308],
       [0.324, 0.676],
       [0.612, 0.388],
       [0.924, 0.076],
       [0.78 , 0.22 ],
       [0.66 , 0.34 ],
       [0.   , 1.   ],
       [0.876, 0.124],
       [0.   , 1.   ],
       [0.38 , 0.62 ],
       [0.5

In [33]:
# setup threshold
threshold = 0.8

In [34]:
# calc maximum predicted probability for each row (compound) and compare to the threshold
da = np.amax(pred_prob, axis=1) > threshold

In [35]:
da

array([ True, False, False,  True, False, False,  True,  True,  True,
        True, False,  True, False,  True, False, False,  True,  True,
        True,  True, False,  True,  True, False, False, False, False,
       False, False, False,  True,  True,  True, False, False, False,
        True, False, False,  True,  True,  True, False, False, False,
       False, False, False,  True, False, False,  True, False, False,
       False,  True, False,  True,  True,  True,  True,  True,  True,
       False,  True])

In [36]:
# calc statistics
print("Accuracy = ", accuracy_score(np.asarray(y_ts)[da], pred_rf[da]))
print("MCC = ", matthews_corrcoef(np.asarray(y_ts)[da], pred_rf[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(y_ts)[da], pred_rf[da]))

Accuracy =  0.9032258064516129
MCC =  0.7947695843356161
Kappa =  0.7928730512249443


In [37]:
# calc coverage
sum(da) / len(da)

0.47692307692307695

#### Build SVM model

In [38]:
# create grid search dictionary
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [39]:
# setup model building
svm = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid, n_jobs=2, cv=cv, verbose=1)

In [40]:
# run model building
svm.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVC(probability=True), n_jobs=2,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [41]:
svm.best_score_

0.7540723981900452

In [42]:
svm.best_params_

{'C': 100, 'gamma': 1e-05}

In [43]:
# save model
joblib.dump(svm, "data/logBB_svm_morgan.pkl", compress=3)

['data/logBB_svm_morgan.pkl']

In [44]:
# predict logBB for the test set compounds
pred_svm = svm.predict(x_ts)

In [45]:
pred_svm

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1])

In [46]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred_svm))
print("MCC = ", matthews_corrcoef(y_ts, pred_svm))
print("Kappa = ", cohen_kappa_score(y_ts, pred_svm))

Accuracy =  0.7846153846153846
MCC =  0.5525513059108836
Kappa =  0.5417925478348439


In [47]:
# estimate applicability domain and calc stat
pred_prob = svm.predict_proba(x_ts)

In [48]:
da = np.amax(pred_prob, axis=1) > threshold

In [49]:
print("Accuracy = ", accuracy_score(np.asarray(y_ts)[da], pred_svm[da]))
print("MCC = ", matthews_corrcoef(np.asarray(y_ts)[da], pred_svm[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(y_ts)[da], pred_svm[da]))
print("Coverage = ", sum(da) / len(da))

Accuracy =  0.8571428571428571
MCC =  0.7071067811865475
Kappa =  0.7058823529411764
Coverage =  0.5384615384615384


### build the third model (GBM) compute consensus predictions from RF, and SVM models

In [50]:
# setup model building
param_grid = {"n_estimators": [100, 200, 300, 400, 500]}
gbm = GridSearchCV(GradientBoostingClassifier(subsample=0.5, max_features=0.5), 
                   param_grid, n_jobs=2, cv=cv, verbose=1)

In [51]:
# run model building
gbm.fit(x_tr, y_tr)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=GradientBoostingClassifier(max_features=0.5,
                                                  subsample=0.5),
             n_jobs=2, param_grid={'n_estimators': [100, 200, 300, 400, 500]},
             verbose=1)

In [52]:
gbm.best_score_

0.7774509803921568

In [53]:
gbm.best_params_

{'n_estimators': 100}

In [54]:
pred_gbm = gbm.predict(x_ts)

In [55]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred_gbm))
print("MCC = ", matthews_corrcoef(y_ts, pred_gbm))
print("Kappa = ", cohen_kappa_score(y_ts, pred_gbm))

Accuracy =  0.7230769230769231
MCC =  0.4245166957273495
Kappa =  0.4236453201970444


#### consensus model

In [56]:
pred_c = 1 * (((pred_rf + pred_svm + pred_gbm) / 3) >= 0.5)

In [57]:
pred_c

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1])

In [58]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred_c))
print("MCC = ", matthews_corrcoef(y_ts, pred_c))
print("Kappa = ", cohen_kappa_score(y_ts, pred_c))

Accuracy =  0.7384615384615385
MCC =  0.45883146774112354
Kappa =  0.4585987261146497


### Add to Morgan fingerprints some other descriptors and look at the model performance

In [59]:
# calc some descriptors
descr = []
for m in mols:
    descr.append([Descriptors.MolLogP(m),
                  Descriptors.TPSA(m),
                  Descriptors.NHOHCount(m),
                  Descriptors.NOCount(m),
                  Descriptors.NumHAcceptors(m),
                  Descriptors.NumHDonors(m),
                  Descriptors.NumRotatableBonds(m),
                  Descriptors.NumHeteroatoms(m),
                  Descriptors.FractionCSP3(m)])
descr = np.asarray(descr)

In [60]:
descr.shape

(321, 9)

In [61]:
# add them to morgan fingerprints
x = np.concatenate((x, descr), axis=1)

In [62]:
x.shape

(321, 2057)

In [63]:
# randomly select 20% of compounds as test set
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.20, random_state=seed)

In [64]:
scale = StandardScaler().fit(x_tr)
x_tr = scale.transform(x_tr)

In [65]:
# create grid search dictionary
param_grid = {"max_features": [x_tr.shape[1] // 10, x_tr.shape[1] // 7, x_tr.shape[1] // 5, x_tr.shape[1] // 3], 
              "n_estimators": [100, 250, 500]}

In [66]:
# setup model building
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [67]:
# run model building
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=2,
             param_grid={'max_features': [205, 293, 411, 685],
                         'n_estimators': [100, 250, 500]},
             verbose=1)

In [68]:
m.best_score_

0.8202865761689291

In [69]:
# scale descriptors of the test set compounds
x_ts = scale.transform(x_ts)

In [70]:
# predict logBB for the test set compounds
pred = m.predict(x_ts)

In [71]:
pred

array([0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1])

In [72]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred))
print("MCC = ", matthews_corrcoef(y_ts, pred))
print("Kappa = ", cohen_kappa_score(y_ts, pred))

Accuracy =  0.8769230769230769
MCC =  0.7473270325025954
Kappa =  0.7410358565737052


In [73]:
# estimate applicability domain and calc stat
pred_prob = m.predict_proba(x_ts)

In [74]:
da = np.amax(pred_prob, axis=1) > threshold

In [75]:
print("Accuracy = ", accuracy_score(np.asarray(y_ts)[da], pred[da]))
print("MCC = ", matthews_corrcoef(np.asarray(y_ts)[da], pred[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(y_ts)[da], pred[da]))
print("Coverage = ", sum(da) / len(da))

Accuracy =  0.9444444444444444
MCC =  0.8864052604279183
Kappa =  0.88
Coverage =  0.5538461538461539


The model has a better accuracy. Added descritors improved the model predictivity.

#### Let's try to analyse which variables are the most important in the model

In [76]:
# rebuild RF model manually using best parameters to be able to extract additional information from the model
rf = RandomForestClassifier(n_estimators=m.best_params_["n_estimators"], 
                           max_features=m.best_params_["max_features"],
                           random_state=seed)
rf.fit(x_tr, y_tr)

RandomForestClassifier(max_features=293, n_estimators=500, random_state=42)

In [77]:
imp = rf.feature_importances_

In [78]:
imp

array([0.        , 0.00211988, 0.00034081, ..., 0.01004565, 0.03644454,
       0.02356641])

In [79]:
indices = np.argsort(imp)[::-1]

print("Feature ranking:")

# print top 10 features
for i in range(10):
    print("%d. feature %d (%f)" % (i + 1, indices[i], imp[indices[i]]))

Feature ranking:
1. feature 2049 (0.118598)
2. feature 2051 (0.055542)
3. feature 2048 (0.048918)
4. feature 2055 (0.036445)
5. feature 650 (0.035974)
6. feature 2052 (0.033922)
7. feature 2056 (0.023566)
8. feature 2050 (0.019445)
9. feature 2053 (0.017630)
10. feature 807 (0.012406)


2049 - MolLogP  
2050 - TPSA(m)  
2051 - NHOHCount  
2052 - NOCount 
2053 - NumHAcceptors  
2054 - NumHDonors  
2055 - NumRotatableBonds  
2056 - NumHeteroatoms  
2057 - FractionCSP3

features with numbers 1-2048 are different Morgan fingerprints  